# Python и интернет. Модуль requests

**План**:

1. Запросы
2. Requests
3. Регулярные выражения
4. bs4 и почему так лучше
5. Задание на семинар

## Как выкачать интернет
Современный Интернет предоставляет лингвистам большое количество языковых данных: электронные газеты и журналы, блоги, форумы, социальные сети и т.д. Например, можно найти в сети много-много текстов и собрать корпус, или найти все газетные статьи и блог-посты про какую-нибудь корпорацию и проанализировать тональность сообщений. Сейчас мы научимся заниматься выкачиванием страниц из интернета с помощью Python.

Для скачивания HTML-страниц в питоне есть несколько библиотек **urllib** и **requests**

## Requests

Можно почитать еще [тут](https://realpython.com/python-requests/)


Допустим, мы хотим скачать главную страницу Хабрахабра.

На самом деле, когда мы хотим открыть какую-то страницу в интернете, наш браузер отправляет на сервер **запрос** ("Привет, сервер! я хочу код страницы по вот такому адресу!"), а сервер затем отправляет ответ ("Привет! Вот код страницы: ...").
Чтобы получить страницу через питон, нужно сформировать **запрос** на сервер так же, как это делает браузер:

In [16]:
import requests

In [2]:
response = requests.get("https://habr.com/ru/")

В response теперь лежит отет сервера. Это не прост html-код страницы, а еще дополнительная информация. Если мы просто выведем этот response, нам покажется только код (200 - все ок).

In [3]:
response

<Response [200]>

In [5]:
response.status_code

200

А вот в атрибуте text уже лежит html-код

In [10]:
print(response.text[:210])

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / 


Иногда сайт блокирует запросы, если их посылает не настоящий браузер с пользователем, а какой-то бот (например, так делает Гугл или Википедия). Иногда сайты присылают разные версии страниц, разным браузерам.  
По этим причинам полезно бывает писать скрипт, который умеет притворяться то одним, то другим браузером.
Когда мы пытаемся получить страницу в питоне, наш код по умолчанию честно сообщает серверу, что он является программой на питоне. Он говорит что-то вроде "Привет, я Python-urllib/3.5". 
Но можно, например, представиться Мозиллой:

In [13]:
url = 'https://habr.com/ru/'  # адрес страницы, которую мы хотим скачать
user_agent = 'Mozilla/5.0 (Windows NT 6.1; Win64; x64)'  # хотим притворяться браузером

response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

Или использовать специальную библиотеку

In [ ]:
from fake_useragent import UserAgent

In [19]:
user_agent = UserAgent().chrome
user_agent

'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/33.0.1750.517 Safari/537.36'

In [20]:
response = requests.get("https://habr.com/ru/", headers={'User-Agent':user_agent})

In [37]:
print(response.text[:300])

<!DOCTYPE html>
<html lang="ru" class="no-js">
  <head>
    <meta http-equiv="content-type" content="text/html; charset=utf-8" />
<meta content='width=1024' name='viewport'>
<title>Лучшие публикации за сутки / Хабр</title>

  <meta name="description" content="Лучшие публикации за последние 24 часа" 


Ура, всё на месте!

Но что всё это значит? Что такое html и как вообще из него доставать какую-то информацию?

Ответ: по **тегам**! Например, в куске html сверху есть теги `<title> </title>` (теги всегда обрамляют с двух сторон то, что находится под этим тегом). В `<title>` в данном случае лежит заголовок этой интернет-страницы.

Существует несколько вариантов, как достать что-то из определенного тега, например, достать заголовок:

1. регулярные выражения
2. специальные библиотеки питона, например, BeautifulSoup (bs4) или lxml.

## Вариант 1: регулярные выражения

Это на самом деле плохой вариант и сейчас стараются так не делать, но иногда бывает нужно и так проще.

Предположим, что мы хотим выкачивать заголовки статей с главной страницы Хабрахабра. Код страницы у нас уже есть, но как из него что-то вытащить. Для начала нужно посмотреть в исходник (view-source:https://habr.com/ru/) и заметить, что заголовки хранятся в тэге **h2** с классом **post__title**. Заголовок выглядит примерно так:

<h2 class="post__title">
    <a href="https://habr.com/ru/company/recognitor/blog/474674/" class="post__title_link">Машинное зрение и медицина</a>
  </h2>
  
Видите, маркдаун понимает html код

А вот и сам html код (это запускать нельзя и не получится, потому что это код "языка" html): 

In [22]:
import re

Чтобы вытащить все такие заголовки постов, воспользуемся регулярным выражением.

In [24]:
html_content = response.text

In [25]:
regex_post_title = re.compile('<h2 class="post__title">(.*?)</h2>', flags= re.DOTALL)
titles = regex_post_title.findall(html_content)

In [26]:
print(len(titles))

20


In [27]:
print(titles[:3])

['\n    <a href="https://habr.com/ru/post/526034/" class="post__title_link">Кейс: как мы на яхте бортовой компьютер заменили</a>\n  ', '\n    <a href="https://habr.com/ru/post/526000/" class="post__title_link">Точные и быстрые вычисления для чисел с плавающей точкой на примере функции синуса. Введение и часть 1</a>\n  ', '\n    <a href="https://habr.com/ru/post/526024/" class="post__title_link">Функциональное программирование на TypeScript: полиморфизм родов высших порядков</a>\n  ']


In [28]:
new_titles = []
regex_tag = re.compile('<.*?>', re.DOTALL)
regex_space = re.compile('\s{2,}', re.DOTALL)
for t in titles:
    clean_t = regex_space.sub("", t)
    clean_t = regex_tag.sub("", clean_t)
    new_titles.append(clean_t)
for t in new_titles:
    print(t)

Кейс: как мы на яхте бортовой компьютер заменили
Точные и быстрые вычисления для чисел с плавающей точкой на примере функции синуса. Введение и часть 1
Функциональное программирование на TypeScript: полиморфизм родов высших порядков
Делаем из Power Mac G4 универсальный и дешевый ретро-компьютер
Чем биоинформатика отличается от вычислительной биологии — краткое введение
Мама, я сделал Хабр
Дайджест свежих материалов из мира фронтенда за последнюю неделю №439 (26 — 31 октября 2020)
Часть 3. Dракоши. Раса Тупиков или стохастическая модель мультиагентной системы
Топ-10 инструментов IoT-разработки в 2020
Высокие технологии
Как я решил сделать игру, вдохновившись Heroes of Might and Magic, и потерял деньги (часть 3)
Играем в Doom в среде VMware ESXi на Raspberry Pi
Я станцевал бы для тебя Menuet
Решение проблемы обеспечения доступности модального окна для людей с ограниченными возможностями
Грядущий кошмар синдрома Кесслера
Zabbix под замком: включаем опции безопасности компонентов Zabbix дл

In [30]:
import html

In [31]:
print(html.unescape('текст1&nbsp;текст2'))

текст1 текст2


### Некоторые объяснения про регулярные выражения (Лирическое отступление)

* Что такое `re.compile`? <br><br>
Грубо говоря, `compile()` позволяет запомнить регулярное выражение и использовать его несколько раз. Суть в том, что перед тем как прогнать регулярку через строку, питон должен ее "скомпилировать" - превратить **строку** с регулярным выражением в специальный **объект**.<br>
Строчка `re.search(..., ...)` сначала компилирует регулярное выражение, а потом выполняет поиск. Если нужно поискать что-то один раз, то такая строчка очень удобна. А если нужно поискать что-то много раз, то получится что одно и то же выражение мы компилируем много раз. А хочется один раз скомпилировать и потом много раз пользоваться. Поэтому пишут так:

In [32]:
text = 'тут текст, внутри которого мы что-то ищем'
regex_name = re.compile('тут регулярное выражение') # скомпилировали
to_search = regex_name.search(text) # теперь можно искать в тексте
to_findall = regex_name.findall(text)  # можно использовать скомпилированное выражение много раз
to_sub = regex_name.sub('на.что.заменить', text) # и так тоже можно использовать

* Что делает `reg_name.sub(..., ...)`?<br><br>
Выражение  `reg_name.sub('на_что_заменить', text)` значит: возьми скомпилированное выражение из переменной `reg_name`, и замени все, что соответствует этому выражению в переменной `text`, на строку `'на_что_заменить'`. Если первый аргумент в этом случае - пустая строка, то все найденные регуляркой куски заменятся на пустую строку, короче говоря, удалятся.<br><br>

* Что такое `re.DOTALL`?<br><br>
Обычно точка в регулярном выражении означает любой символ КРОМЕ символа новой строки.  Чтобы изменить такое поведение, в компиляцию регулярки можно добавить параметры-флаги вот так: `flags = re.DOTALL`, и тогда точка будет ловить вообще любой символ, включая новую строку. Эти флаги слегка меняют поведение функции, вот и все.<br><br>

* Что такое `re.U`?<br><br>
Про эту штуку нужно знать, если вы работаете с регулярками на втором питоне. Дело в том, что во втором питоне по умолчанию выражения типа `\w`, `\W`, `\s` и подобные работают только на строках ASCII, и чтобы они работали на юникодных строках нужно поставить флаг re.U. В третьем питоне все строки и так юникодные, поэтому необходимости в таком флаге нет.

## Вариант 2: BeautifulSoup

Это намного лучше! Код страницы парсится как иерархия тегов (как они есть в html коде, один вложен в другой) и можно искать элементы с помощью разных методов, использовать атрибуты и т.д.

In [33]:
from bs4 import BeautifulSoup

Сначала инициализируем объект BeautifulSoup. Потом применим метод find и в скобочках укажем теги, по которым ищем. У некоторых тегов в html (как и в нашем случае) бывает еще и class и какие-нибудь еще атрибуты. Такие вещи мы задаем словариком.

Этот запрос вернёт нам только первый заголовок. То есть первое вхождение такого тега в нашем html файле.

In [35]:
soup = BeautifulSoup(html_content, 'html.parser')  # инициализируем суп

post = soup.find('h2', {'class': 'post__title'})
print(post.get_text())
print(post.prettify())


Кейс: как мы на яхте бортовой компьютер заменили

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/526034/">
  Кейс: как мы на яхте бортовой компьютер заменили
 </a>
</h2>



Но мы хотим получить все заголовки постов! Метод find_all возвращает массив всех элементов с тегом указанным в скобках. По нему можно итерироваться.

In [36]:
for post in soup.find_all('h2', {'class': 'post__title'})[:3]:
    print(post.get_text())
    print(post.prettify())

    print('-- '*10)  # для красоты


Кейс: как мы на яхте бортовой компьютер заменили

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/526034/">
  Кейс: как мы на яхте бортовой компьютер заменили
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 

Точные и быстрые вычисления для чисел с плавающей точкой на примере функции синуса. Введение и часть 1

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/526000/">
  Точные и быстрые вычисления для чисел с плавающей точкой на примере функции синуса. Введение и часть 1
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 

Функциональное программирование на TypeScript: полиморфизм родов высших порядков

<h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/526024/">
  Функциональное программирование на TypeScript: полиморфизм родов высших порядков
 </a>
</h2>

-- -- -- -- -- -- -- -- -- -- 


## Задание на семинар 1

А что если мы хотим зайти еще глубже по дереву тегов и, например, для каждого заголовка поста найти никнейм юзера, который написал этот пост, и время написания поста?

Для этого надо снова зайти в просмотор кода страницы и увидеть, что там просиходит что-то такое:

(Заодно обратите внимание, как пишутся комменты в html)

In [ ]:
<li class="content-list__item content-list__item_post shortcuts_item" id="post_526034">
                
    <article class="post post_preview" lang="ru">
  <header class="post__meta">
    <a href="https://habr.com/ru/users/Autonomnoe/" class="post__user-info user-info" title="Автор публикации">
        <img src="//habrastorage.org/getpro/habr/avatars/35b/44f/fe7/35b44ffe76e9518b5d07c5748372937f.png" width="24" height="24" class="user-info__image-pic user-info__image-pic_small">
      <span class="user-info__nickname user-info__nickname_small">Autonomnoe</span>
    </a>

    <span class="post__time">вчера в 23:55</span>

    
  </header>

  <h2 class="post__title">
    <a href="https://habr.com/ru/post/526034/" class="post__title_link">Кейс: как мы на яхте бортовой компьютер заменили</a>
  </h2>


## Задание на семинар 2

Скачать главную страницу Яндекс.Погоды и
    
- а) распечатать сегодняшнюю температуру и облачность
- б) распечатать время восхода и заката
- в) погоду на завтра

## Хорошая статья про это все

[https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/](https://sysblok.ru/courses/obkachka-sajtov-svoimi-rukami-razbiraemsja-s-html/)